<a href="https://colab.research.google.com/github/mike-ferguson/benford/blob/main/benford_verifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [237]:
# import or generate a dataset:

import pandas as pd, numpy as np
import random
import math 
from collections import Counter
from scipy import stats

from google.colab import files




def main(usr_mode, columns):
  p_vals = []
  ts = []
  col_name = []
  statuses = []
  datasets = []

  # read in the dataset:
  dataset = 'sb_loan.csv'
  data = pd.read_csv(dataset, encoding= 'unicode_escape')
  print(data.columns)
  
 

  for column in columns:
#
     # use this to find the columns you want to use:
    col_wanted = column


    # remove NA values or values not a number.
    df = data[data[col_wanted].notna()]
    df = df[pd.to_numeric(df[col_wanted], errors='coerce').notnull()]
    df_final = df[col_wanted]

    # casts entries to floats and make final list
    numbers = list(df_final)
    numbers = [float(i) for i in numbers]
    numbers = [abs(i) for i in numbers]
    numbers = [x for x in numbers if x > 0.0]


    #----------------------------------------------------------------------------

    # look at the data to see properties, like normality:

    from matplotlib import pyplot
    import statistics

    #pyplot.hist(numbers)
    #pyplot.show()

    mean = statistics.mean(numbers)
    std = statistics.stdev(numbers)
    median = statistics.median(numbers)
    #print("Mean: ", mean)
    #print("Standard Deviation: ", std)


    # is mean greater then median?
    if mean <= median:
      print(column)
      print("The mean is not greateer then the median. Your distribution might be" + 
            "off; Benford's Law might not apply...")
      
      # fail safe- the program will exit if this fails. You can then look and see 
      # what columns fail and delete them from analysis.
      # return
      


    #----------------------------------------------------------------------------
    # process the raw list:
    sig_list = ([x // (10**math.floor(math.log10(x))) for x in numbers])
    #print(sig_list[0:50])

    # get Benford probabilities:
    def benford (digit):
      return math.log10((digit + 1)/ digit)


    #----------------------------------------------------------------------------
    # calculate meta-statistics:

    actual_freqs = Counter(sig_list)

    limit = len(numbers)

    predicted = []
    for i in range(1, 10):
      pred_num = benford(i) * limit
      predicted.append(round(pred_num, 4))

    benford_list = [1,2,3,4,5,6,7,8,9]

    predicted_freqs = dict(zip(benford_list, predicted))

    df = pd.DataFrame({'Predicted Values':pd.Series(predicted_freqs),'Actual Values':pd.Series(actual_freqs)})


    df["Predicted Percent"] = (df['Predicted Values'] / limit) * 100
    df["Actual Percent"] = (df['Actual Values']  / limit) * 100

    # df["Actual Percent"] = df['Actual Percent'].dropna(0)
    # df["Actual Values"] = df["Actual Values"].dropna(0)
    df = df.fillna(0)
    #print(df)

    actual_props = list(df["Actual Percent"])
    predicted_props = list(df["Predicted Percent"])

    #----------------------------------------------------------------------------

    # create test statistic:
    chi_squared = 0
    for i in range(0,9):
      error = ((actual_props[i] - predicted_props[i]) ** 2) / predicted_props[i]
      chi_squared  = chi_squared + error

    # print(chi_squared)




    #----------------------------------------------------------------------------
    # conduct actual hypthesis test:
    # H0: All digits 1-9 Conform to the predicted Benford's Law Proportion
    # H1: At least 1 digit does not conform to the predictions.


    dof = 8
    value = 1 - stats.chi2.cdf(chi_squared, dof)
    # print( "P value: " ,value)
    # print( "Alpha: " , 0.05)

    if value < 0.05:
      #print("Results are significant at this Alpha Level (0.05). \nThis indicates " + 
            #"very strong evidence that Benford's Law is NOT obeyed. H0 is rejected.")
      status = "Not Obeyed"
    else:
      #print("Results are not statistically significant. \nThis indicates " + 
            #"very strong evidence that Benford's Law IS obeyed. Fail to reject H0.")
      status = "Obeyed"
      
    print(column + " done")
    p_vals.append(value)
    ts.append(chi_squared)
    col_name.append(column)
    statuses.append(status)
    datasets.append(dataset)

  final_df= pd.DataFrame(
    {'Dataset': datasets,
     'Col Name': col_name,
     'Chi_Sq Val': ts, 
     'P-value': p_vals,
     'Pass/Fail': statuses,
    })
  print(final_df)
  name = dataset.strip(".csv")
  save_name = name + '_results.csv'
  final_df.to_csv(save_name)
  files.download(save_name)

In [238]:

columns = [
       'NoEmp',
       'CreateJob', 'RetainedJob'
        ]
main("muliti", columns)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')
NoEmp done
CreateJob done
RetainedJob done
       Dataset     Col Name  Chi_Sq Val   P-value Pass/Fail
0  sb_loan.csv        NoEmp    3.175600  0.922859    Obeyed
1  sb_loan.csv    CreateJob   10.732387  0.217328    Obeyed
2  sb_loan.csv  RetainedJob    3.617698  0.889866    Obeyed


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Criteria for selection**

1. Passed Median > mode test 
2. Numbers seamed reasonable
3. No pricing data
4. Consisted of a single file
5. No Repeat Columns
6. Top 10 in Search
7. Consist of Numerical Data



**Sales Datasets**

1. Video Game Sales:  **2/5 Obeyed**
2. Avocado Prices : **8/8 Obeyed**
3. House Sales in King Country: **0/6 Obeyed**
4. Video Game Sales with Ratings: **2/2 Obeyed**
6. Supermarket Sales: **4/5 Obeyed**
7. NYC Property Sales: **1/6 Obeyed**
8. Sample Sales Data: **0/2 Obeyed**
9. Fish Market: **1/6 Obeyed**
10. Historical Sales and Active Inventory: **1/2 Obeyed** 
**Total Obeyed: 19/46 or 41.3%**


**Top Datasets**

1. Credit Card Fraud Detection: **28/30 Obeyed**
5. Suicide Rates Overview 1985-2016: **4/4 Obeyed**
6. Bitcoin Histroical Data: **2/2 Obeyed**
9. Pima Indians Diabetes Database: **3/8 Obeyed**
10. IBM Analytics: **3/11 Obeyed** 

**Total Obeyed: 40/55 = 72.72%**


**Disbursement Datasets**

1. Should This Loan be Approved or Denied?: **3/3 Obeyed**
2. Loan Default: **9/19 Obeyed**
3. NYS New York State Budget and Actuals: **26/26 Obeyed**
4. Loan Disbursement: **0/1 Obeyed**
5. Indian Loan Disbursement Data: **7/17 Obeyed**

**Total Obeyed: 45/66


Grand Total Obeyed: 

** 19 Total datasets**



**Notable Findings**

1.   'Ratings' Data was not good- consistently failed the mean/median test for supermarket, video game sales



**Sales Links to Datasets:**
1. Video Game Sales:  https://www.kaggle.com/gregorut/videogamesales
2. Avocado Prices : https://www.kaggle.com/neuromusic/avocado-prices
3. House Sales in King County: https://www.kaggle.com/harlfoxem/housesalesprediction
4. Video Game Sales with Ratings: https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings

6. Supermarket Sales: https://www.kaggle.com/aungpyaeap/supermarket-sales
7. NYC Property Sales: https://www.kaggle.com/new-york-city/nyc-property-sales
8. Sample Sales Data: https://www.kaggle.com/kyanyoga/sample-sales-data
9. Fish Market: https://www.kaggle.com/aungpyaeap/fish-market
10. Historical Sales and Active Inventory: https://www.kaggle.com/flenderson/sales-analysis


**Links to Top Datasets**

1. Credit Card Fraud Detection: https://www.kaggle.com/mlg-ulb/creditcardfraud

5. Suicide Rates Overview 1985-2016: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016
6. Bitcoin Histroical Data: https://www.kaggle.com/mczielinski/bitcoin-historical-data


9. Pima Indians Diabetes Database: https://www.kaggle.com/uciml/pima-indians-diabetes-database
10. IBM HR Analytics Employee Attrition & Performance


**Datasets eligible but not used (lack of usable data)**
2. Heart Disease UCI : https://www.kaggle.com/ronitf/heart-disease-uci 
3. European Soccer Database: https://www.kaggle.com/hugomathien/soccer 
4. FIFA 19 Complete Player Dataset: https://www.kaggle.com/karangadiya/fifa19 
7. Netflix Movies and TV Shows: https://www.kaggle.com/shivamb/netflix-shows
8. Global Terrorism Database: https://www.kaggle.com/START-UMD/gtd
10. Pokemon with Stats: https://www.kaggle.com/abcsds/pokemon
5. Used Car Dataset: https://www.kaggle.com/austinreese/craigslist-carstrucks-data

**Disbursement Datasets Links**
1. Should This Loan be Approved or Denied?:https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied
2. Loan Default: https://www.kaggle.com/lampubhutia/loandefault-ltfs-avml-finhack
3. NYS New York State Budget and Actuals: https://www.kaggle.com/new-york-state/nys-new-york-state-budget-and-actuals
4. Loan Disbursement: https://www.kaggle.com/sonnykurniawan/loan-disbursement
5. Indian Loan Disbursement Data: https://www.kaggle.com/iamprateek/indian-car-loan-disbursement-data
